In [49]:
import pandas as pd
from sklearn import linear_model
import cv2 as cv
import numpy as np

In [33]:
data_path_prefix = "D:/Uni Stuff/IP/Data/"
image_path_prefix = data_path_prefix + "Blimp Images/Raw/image_"

train_test_split = 0.75

desired_image_size = 16

In [34]:
bbox_data = pd.read_csv(data_path_prefix + "bbox data.csv")
pose_data = pd.read_csv(data_path_prefix + "blimp poses.csv")

In [51]:
def cutout_image(image, bbox):
    width = image.shape[1]
    height = image.shape[0]

    center_x = bbox['cent_x'] * width
    center_y = bbox['cent_y'] * height

    bbox_width = bbox['width'] * width
    bbox_height = bbox['height'] * height

    min_x = np.clip(int(center_x - (bbox_width / 2)), 0, width)
    max_x = np.clip(int(center_x + (bbox_width / 2)), 0, width) 

    min_y = np.clip(int(center_y - (bbox_height / 2)), 0, height)
    max_y = np.clip(int(center_y + (bbox_height / 2)), 0, height)

    return image[min_y:max_y, min_x:max_x]

def process_image(image):
    old_size = image.shape[:2]

    size_ratio = float(desired_image_size) / max(old_size)

    new_size = tuple([int(x * size_ratio) for x in old_size])

    resized_image = cv.resize(image, (new_size[1], new_size[0]), interpolation=cv.INTER_AREA)

    delta_w = desired_image_size - new_size[1]
    delta_h = desired_image_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)

    color = [0, 0, 0]
    padded_image = cv.copyMakeBorder(resized_image, top, bottom, left, right, cv.BORDER_CONSTANT, value=color)

    return padded_image

images = []

for i in range(len(pose_data)):
    img = cv.imread(image_path_prefix + str(i) + ".png")
    cropped = cutout_image(img, bbox_data.iloc[i])
    padded = process_image(cropped)

    images.append(padded)

In [ ]:
train_no = int(len(pose_data) * train_test_split)

train_X = bbox_data.iloc[0:train_no]
train_y = pose_data[0:train_no]

test_X = bbox_data.iloc[train_no:len(bbox_data)]
test_y = pose_data.iloc[train_no:len(pose_data)]

In [20]:
regr = linear_model.LinearRegression()
regr.fit(train_X, train_y)

LinearRegression()

In [24]:
print("Actual: " + str(test_y.iloc[0]))

predicted_pose = regr.predict([test_X.iloc[0]])
print("Predicted: " + str(predicted_pose))

Actual: pos_x    54.913242
pos_y     4.765702
pos_z   -17.306404
rot_x    -0.009107
rot_y    -0.184368
rot_z    -2.365281
Name: 225, dtype: float64
Predicted: [[ 4.99183383e+01  3.94797454e+00 -1.15908687e+01 -4.29642171e-04
   2.58841504e-02  1.08613312e-01]]


c:\Users\frase\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
